### Week 6, Day 2

We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [22]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [23]:
from accounts import Account

In [24]:
account = Account.get("Ram")
account

Account(name='ram', balance=9810.622, strategy='', holdings={'APPL': 6}, transactions=[3 shares of APPL at 21.042 each., 3 shares of APPL at 42.084 each.], portfolio_value_time_series=[('2025-10-24 16:30:01', 9978.874), ('2025-10-24 16:30:10', 10011.874), ('2025-10-24 16:34:36', 10152.622), ('2025-10-24 16:34:42', 10314.622), ('2025-10-24 16:45:41', 10050.622), ('2025-10-24 16:45:47', 10368.622), ('2025-10-24 16:47:35', 10182.622)])

In [25]:
account.buy_shares("APPL", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ram", "balance": 9516.034, "strategy": "", "holdings": {"APPL": 9}, "transactions": [{"symbol": "APPL", "quantity": 3, "price": 21.042, "timestamp": "2025-10-24 16:30:01", "rationale": "Because this bookstore website looks promising"}, {"symbol": "APPL", "quantity": 3, "price": 42.084, "timestamp": "2025-10-24 16:34:36", "rationale": "Because this bookstore website looks promising"}, {"symbol": "APPL", "quantity": 3, "price": 98.196, "timestamp": "2025-10-24 17:05:15", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-10-24 16:30:01", 9978.874], ["2025-10-24 16:30:10", 10011.874], ["2025-10-24 16:34:36", 10152.622], ["2025-10-24 16:34:42", 10314.622], ["2025-10-24 16:45:41", 10050.622], ["2025-10-24 16:45:47", 10368.622], ["2025-10-24 16:47:35", 10182.622], ["2025-10-24 17:05:15", 10020.034]], "total_portfolio_value": 10020.034, "total_profit_loss": 20.03399999999965}'

In [8]:
account.report()

'{"name": "ram", "balance": 9810.622, "strategy": "", "holdings": {"APPL": 6}, "transactions": [{"symbol": "APPL", "quantity": 3, "price": 21.042, "timestamp": "2025-10-24 16:30:01", "rationale": "Because this bookstore website looks promising"}, {"symbol": "APPL", "quantity": 3, "price": 42.084, "timestamp": "2025-10-24 16:34:36", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-10-24 16:30:01", 9978.874], ["2025-10-24 16:30:10", 10011.874], ["2025-10-24 16:34:36", 10152.622], ["2025-10-24 16:34:42", 10314.622]], "total_portfolio_value": 10314.622, "total_profit_loss": 314.6219999999994}'

In [9]:
account.list_transactions()

[{'symbol': 'APPL',
  'quantity': 3,
  'price': 21.042,
  'timestamp': '2025-10-24 16:30:01',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'APPL',
  'quantity': 3,
  'price': 42.084,
  'timestamp': '2025-10-24 16:34:36',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [10]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()


In [11]:
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [12]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ram and my account is under the name Ram. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [13]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Ram, your current cash balance is approximately $9,810.62. In terms of holdings, you own 6 shares of AAPL (Apple Inc.). Is there anything specific you would like to do with your account or any other information you need?

### Now let's build our own MCP Client

In [14]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None), Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None), Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', 'ty

In [15]:
request = "My name is Ram and my account is under the name Ram. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Your account balance is $9,810.62. If you need any other information or assistance, please let me know!

In [17]:
context = await read_accounts_resource("Ram")
print(context)

{"name": "ram", "balance": 9810.622, "strategy": "", "holdings": {"APPL": 6}, "transactions": [{"symbol": "APPL", "quantity": 3, "price": 21.042, "timestamp": "2025-10-24 16:30:01", "rationale": "Because this bookstore website looks promising"}, {"symbol": "APPL", "quantity": 3, "price": 42.084, "timestamp": "2025-10-24 16:34:36", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-10-24 16:30:01", 9978.874], ["2025-10-24 16:30:10", 10011.874], ["2025-10-24 16:34:36", 10152.622], ["2025-10-24 16:34:42", 10314.622], ["2025-10-24 16:45:41", 10050.622]], "total_portfolio_value": 10050.622, "total_profit_loss": 50.62199999999939}


In [26]:
from accounts import Account
Account.get("Ram").report()

'{"name": "ram", "balance": 9516.034, "strategy": "", "holdings": {"APPL": 9}, "transactions": [{"symbol": "APPL", "quantity": 3, "price": 21.042, "timestamp": "2025-10-24 16:30:01", "rationale": "Because this bookstore website looks promising"}, {"symbol": "APPL", "quantity": 3, "price": 42.084, "timestamp": "2025-10-24 16:34:36", "rationale": "Because this bookstore website looks promising"}, {"symbol": "APPL", "quantity": 3, "price": 98.196, "timestamp": "2025-10-24 17:05:15", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-10-24 16:30:01", 9978.874], ["2025-10-24 16:30:10", 10011.874], ["2025-10-24 16:34:36", 10152.622], ["2025-10-24 16:34:42", 10314.622], ["2025-10-24 16:45:41", 10050.622], ["2025-10-24 16:45:47", 10368.622], ["2025-10-24 16:47:35", 10182.622], ["2025-10-24 17:05:15", 10020.034], ["2025-10-24 17:08:31", 10344.034]], "total_portfolio_value": 10344.034, "total_profit_loss": 344.03399999999965}'

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Make your own MCP Server! Make a simple function to return the current Date, and expose it as a tool so that an Agent can tell you today's date.<br/>Harder optional exercise: then make an MCP Client, and use a native OpenAI call (without the Agents SDK) to use your tool via your client.
            </span>
        </td>
    </tr>
</table>